In [27]:
#from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.optimizers import Adam
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import os
import pandas as pd
import datetime as dt

from functools import partial

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.utils import class_weight

from tensorflow.python.keras import backend as K

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

**Data manipulation functions**

In [23]:
def format_multiple_bins_for_LSTM(
    dataframe,
    target_column_index,
    history_size,
    target_size, 
    step
):

    data = []
    labels = []
    
    spatial_bins = dataframe.groupby(['lat', 'lon'])
    
    for bin_name, spatial_bin in spatial_bins:
        
        spatial_bin = spatial_bin.sort_index()
        
        spatial_bin = np.array(spatial_bin.values)
        target = spatial_bin[:, target_column_index]
        
        bin_data = []
        bin_labels = []
    
        start_index = history_size
        end_index = len(spatial_bin) - target_size

        for i in range(start_index, end_index):
            indices = range(i - history_size, i, step)
            bin_data.append(spatial_bin[indices])
            bin_labels.append(target[i + target_size])

        data.append(np.array(bin_data))
        labels.append(np.array(bin_labels))

    return data, labels
    

def plot_metrics(history):
    metrics =  ['loss', 'auc', 'f1']
    
    plt.subplots(1,4,figsize=(10.5,3.5))
    
    for n, metric in enumerate(metrics):
        name = metric.replace("_"," ").capitalize()
        plt.subplot(1,3,n+1)
        plt.plot(history.epoch,  history.history[metric], color='royalblue', label='Train')
        plt.plot(history.epoch, history.history['val_'+metric],
             color='royalblue', linestyle="--", label='Val')
        plt.xlabel('Epoch')
        plt.ylabel(name)
        plt.title(name)
        
    if metric == 'loss':
        plt.ylim([0, plt.ylim()[1]])
        
    elif metric == 'auc':
        plt.ylim([0.8,1])
        
    else:
        #plt.ylim([0,1])
        plt.legend()
        
    plt.tight_layout()
    plt.savefig('../figures/parallel_LSTM_learning_curves.png', bbox_inches='tight')
    
def f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    
    return f1_val

In [3]:
data_file = '../data/training_data/1992-2015_training_data_added_features.csv'

# Datatypes for dataframe loading
dtypes = {
    'lat': float,
    'lon': float,
    'weather_bin_year': int,
    'weather_bin_month': int,
    'weather_bin_day': int,
    'air.2m': float,
    'apcp': float,
    'rhum.2m': float,
    'dpt.2m': float,
    'pres.sfc': float,
    'uwnd.10m': float,
    'vwnd.10m': float,
    'veg': float,
    'vis': float,
    'ignition': float,
    'mean.air.2m': float,
    'mean.apcp': float,
    'mean.rhum.2m': float,
    'mean.dpt.2m': float,
    'mean.pres.sfc': float,
    'mean.uwnd.10m': float,
    'mean.vwnd.10m': float,
    'mean.veg': float,
    'mean.vis': float,
    'max.air.2m': float,
    'max.apcp': float,
    'max.rhum.2m': float,
    'max.dpt.2m': float,
    'max.pres.sfc': float,
    'max.uwnd.10m': float,
    'max.vwnd.10m': float,
    'max.veg': float,
    'max.vis': float,
    'min.air.2m': float,
    'min.apcp': float,
    'min.rhum.2m': float,
    'min.dpt.2m': float,
    'min.pres.sfc': float,
    'min.uwnd.10m': float,
    'min.vwnd.10m': float,
    'min.veg': float,
    'min.vis': float,
    'total_fires': float

}

# Features to use during training 
features = [
    'lat',
    'lon',
    'weather_bin_month',
    'veg',
    'ignition',
    'mean.air.2m',
    'mean.apcp',
    'mean.rhum.2m',
    'mean.dpt.2m',
    'mean.pres.sfc',
    'mean.uwnd.10m',
    'mean.vwnd.10m',
    'mean.vis',
    'max.air.2m',
    'max.apcp',
    'max.rhum.2m',
    'max.dpt.2m',
    'max.pres.sfc',
    'max.uwnd.10m',
    'max.vwnd.10m',
    'max.vis',
    'min.air.2m',
    'min.apcp',
    'min.rhum.2m',
    'min.dpt.2m',
    'min.pres.sfc',
    'min.uwnd.10m',
    'min.vwnd.10m',
    'min.vis',
    'total_fires'
]

features_to_scale = [
    'lat',
    'lon',
    'veg',
    'mean.air.2m',
    'mean.apcp',
    'mean.rhum.2m',
    'mean.dpt.2m',
    'mean.pres.sfc',
    'mean.uwnd.10m',
    'mean.vwnd.10m',
    'mean.vis',
    'max.air.2m',
    'max.apcp',
    'max.rhum.2m',
    'max.dpt.2m',
    'max.pres.sfc',
    'max.uwnd.10m',
    'max.vwnd.10m',
    'max.vis',
    'min.air.2m',
    'min.apcp',
    'min.rhum.2m',
    'min.dpt.2m',
    'min.pres.sfc',
    'min.uwnd.10m',
    'min.vwnd.10m',
    'min.vis',
    'total_fires'
]

In [4]:
raw_data = pd.read_csv(data_file, index_col=0, parse_dates=True, dtype=dtypes)

In [5]:
# Pull out columns of intrest
data = raw_data[features]

In [6]:
# One hot encode month
column_names = [
    'January',
    'February',
    'March',
    'April',
    'May',
    'June',
    'July',
    'August',
    'Septermber',
    'October',
    'November',
    'December'
]


onehot_encoder = OneHotEncoder(sparse=False)

# Training data
month = np.array(data['weather_bin_month']).reshape(-1, 1)
onehot_month = onehot_encoder.fit_transform(month)

data.drop('weather_bin_month', axis=1, inplace=True)
onehot_month_df = pd.DataFrame(onehot_month, columns=column_names)

onehot_month_df['datetime'] = pd.to_datetime(data.index)
onehot_month_df = onehot_month_df.set_index('datetime')
data = pd.concat([data, onehot_month_df], axis=1)

In [41]:
# Scale data
scaler = StandardScaler()
scaled_features = scaler.fit_transform(data[features_to_scale])
data[features_to_scale] = scaled_features

In [42]:
# Split data up into training, testing and validation sets
test_data = data.tail(int(len(data)*0.1))
leftover_data = data.iloc[:-int(len(data)*0.1)]

validation_data = data.tail(int(len(leftover_data)*0.3))
training_data = data.iloc[:-int(len(leftover_data)*0.3)]

In [43]:
# Split data into list 
target_column_index = 4
past_history = 3
future_target = 1
step = 1

x_train, y_train = format_multiple_bins_for_LSTM(
    training_data, 
    target_column_index, 
    past_history,
    future_target, 
    step,
)

x_validation, y_validation = format_multiple_bins_for_LSTM(
    validation_data, 
    target_column_index, 
    past_history,
    future_target, 
    step,
)

In [ ]:
start_index = (x_train.shape[0] - (x_train.shape[0] % 100))
end_index = x_train.shape[0]

x_train = np.delete(x_train, range(start_index, end_index), axis=0)
y_train = np.delete(y_train, range(start_index, end_index), axis=0)

start_index = (x_validation.shape[0] - (x_validation.shape[0] % 100))
end_index = x_validation.shape[0]

x_validation = np.delete(x_validation, range(start_index, end_index), axis=0)
y_validation = np.delete(y_validation, range(start_index, end_index), axis=0)

In [44]:
sample_sizes = []

for sample in y_train:
    sample_sizes.append(len(sample))
    
smallest_sample = min(sample_sizes)

y_train_reshaped = []

for i in range(smallest_sample):
    y = []
    for j in range(len(y_train)):
        try:
            y.append(y_train[j][i])
        except:
            print("Index out of range")
    
    y_train_reshaped.append(np.array(y))
    
trimmed_x_training = []    
    
for sample in x_train:
    trimmed_sample = sample[-smallest_sample:,:]
    trimmed_x_training.append(trimmed_sample)

In [45]:
sample_sizes = []

for sample in y_validation:
    sample_sizes.append(len(sample))
    
smallest_sample = min(sample_sizes)
y_validation_reshaped = []

for i in range(smallest_sample):
    y = []
    for j in range(len(y_validation)):
        try:
            y.append(y_validation[j][i])
        except:
            print("Index out of range")
    
    y_validation_reshaped.append(np.array(y))
    
trimmed_x_validation = []    
    
for sample in x_validation:
    trimmed_sample = sample[-smallest_sample:,:]
    trimmed_x_validation.append(trimmed_sample)

In [12]:
lstm_units = 90
dense_units = 95
l2_lambda = 0.1
learning_rate = 0.0001

metrics = [
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.FalseNegatives(name='fn'), 
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc'),
    f1
]

In [59]:
inputs = []
LSTMs = []

In [60]:
for i in range(len(trimmed_x_training)):
    inputs.append(
        keras.Input(
            #trimmed_x_training[0].shape[-2:],
            batch_shape=(1,3,30)
        )
    )

In [61]:
for i in range(len(trimmed_x_training)):
    LSTMs.append(keras.layers.LSTM(
        lstm_units,
        stateful=True
    )(inputs[i]))

In [62]:
merged = keras.layers.concatenate(LSTMs)

hidden1 = keras.layers.Dense(
    dense_units,
    bias_initializer=keras.initializers.VarianceScaling(
        scale=1.0,
        mode='fan_in', 
        distribution='normal', 
        seed=None
    ),
    kernel_regularizer=keras.regularizers.l2(l2_lambda),
    activation = 'relu'
)(merged)

hidden2 = keras.layers.Dense(
    dense_units,
    bias_initializer=keras.initializers.VarianceScaling(
        scale=1.0,
        mode='fan_in', 
        distribution='normal', 
        seed=None
    ),
    kernel_regularizer=keras.regularizers.l2(l2_lambda),
    activation = 'relu'
)(hidden1)

output = keras.layers.Dense(
    410,
    activation = 'sigmoid'
)(hidden2)

In [63]:
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
# weight_for_0 = (1 / no_ignition_count)*(total)/2.0 
# weight_for_1 = (1 / ignition_count)*(total)/2.0

# class_weight = {0: weight_for_0, 1: weight_for_1}

In [64]:
class_weights = []
for spatial_bin in y_train:
    total = len(spatial_bin)
    ignition = sum(spatial_bin)
    no_ignition = total - ignition
    class_1_weight = (1 / (ignition + K.epsilon())) * total / 2.0
    class_0_weight = (1 / (no_ignition + K.epsilon())) * total / 2.0
    class_weight = {0: class_0_weight, 1: class_1_weight}
    class_weights.append(class_weight)

In [70]:
model = keras.Model(inputs=inputs, outputs=output)

model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=learning_rate), 
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=metrics
)

model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_821 (InputLayer)          [(1, 3, 30)]         0                                            
__________________________________________________________________________________________________
input_822 (InputLayer)          [(1, 3, 30)]         0                                            
__________________________________________________________________________________________________
input_823 (InputLayer)          [(1, 3, 30)]         0                                            
__________________________________________________________________________________________________
input_824 (InputLayer)          [(1, 3, 30)]         0                                            
____________________________________________________________________________________________

__________________________________________________________________________________________________
lstm_841 (LSTM)                 (1, 90)              43560       input_1161[0][0]                 
__________________________________________________________________________________________________
lstm_842 (LSTM)                 (1, 90)              43560       input_1162[0][0]                 
__________________________________________________________________________________________________
lstm_843 (LSTM)                 (1, 90)              43560       input_1163[0][0]                 
__________________________________________________________________________________________________
lstm_844 (LSTM)                 (1, 90)              43560       input_1164[0][0]                 
__________________________________________________________________________________________________
lstm_845 (LSTM)                 (1, 90)              43560       input_1165[0][0]                 
__________

In [71]:
print(trimmed_x_training[0].shape)
y_train_reshaped = np.array(y_train_reshaped)
print(y_train_reshaped.shape)

(6395, 3, 30)
(6395, 410)


In [72]:
print(trimmed_x_validation[0].shape)
y_validation_reshaped = np.array(y_validation_reshaped)
print(y_validation_reshaped.shape)

(2362, 3, 30)
(2362, 410)


In [ ]:
history = model.fit(
    trimmed_x_training, 
    np.array(y_train_reshaped),
    batch_size=1, 
    epochs=5,
    steps_per_epoch=3000,
    validation_steps=1000,
    validation_data=(trimmed_x_validation, np.array(y_validation_reshaped)),
    class_weight=class_weights
)

Train on 6395 samples, validate on 2362 samples
Epoch 1/5
 916/6395 [===>..........................] - ETA: 15:58 - loss: 637.1275 - tp: 1576.0000 - fp: 7916.0000 - tn: 353574.0000 - fn: 12494.0000 - accuracy: 0.9457 - precision: 0.1660 - recall: 0.1120 - auc: 0.5451 - f1: 0.1134

In [ ]:
plot_metrics(history)